# 4-4. 분류용 데이터 세트를 준비합니다

In [ ]:
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
cancer = load_breast_cancer()

In [ ]:
print(cancer.data.shape, cancer.target.shape)
cancer.data[:3]

In [ ]:
sns.boxplot(data=cancer.data)
plt.xlabel('feature')
plt.ylabel('value')
plt.show()

In [ ]:
cancer.feature_names[[3,13,23]]

In [ ]:
np.unique(cancer.target, return_counts=True)

In [ ]:
212/357

In [ ]:
x = cancer.data
y = cancer.target

# 4-5. 로지스틱 회귀를 위한 뉴런을 만듭니다

![training_test_set](training_test_set.png "training_test_set")

#### 1. 테스트 세트보다 훈련 세트가 더 커야 합니다.
#### 2. 양성, 음성 클래스의 비율이 양 세트에서 균일해야 합니다.
> 학습이 잘못 되거나, 성능 측정이 잘못될 위험성 있음

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

- stratify : 해당 데이터의 subset 비율을 유지(여기서는 음성, 양성 클래스 비율)
- test_size : 테스트 세트의 크기

In [ ]:
print(x_train.shape, x_test.shape)

In [ ]:
np.unique(y_train, return_counts=True)

In [ ]:
170/285

## 로지스틱 회귀 구현하기

In [42]:
class LogisticNeuron:
    def __init__(self):
        self.w = None
        self.b = None
        
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        return z
    
    def backprop(self, x, err):
        w_grad = x * err
        b_grad = 1 * err
        return w_grad, b_grad
    
    def fit(self, x, y, epochs=100):
        self.w = np.ones(x.shape[1]) # x.shape[1] : feature 개수
        self.b = 0
        for i in range(epochs):
            for x_i, y_i in zip(x, y):
                z = self.forpass(x_i)
                a = self.activation(z)
                err = -(y_i - a)
                w_grad, b_grad = self.backprop(x_i, err)
                self.w -= w_grad
                self.b -= b_grad
    
    def activation(self, z): # z is an numpy array
        a = 1 / (1 + np.exp(-z))
        return a
    
    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x]
        a = self.activation(np.array(z))
        return a > 0.5

- \_\_init\_\_: 가중치, 절편 미리 초기화 하지 않음
- forpass: $x \cdot w + b$ 구현
- activation: numpy array를 argument로 받으면 자동으로 function을 모든 element에 적용

In [44]:
neuron = LogisticNeuron()
neuron.fit(x_train, y_train)

C:\Users\skkuj\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in exp


In [45]:
np.mean(neuron.predict(x_test) == y_test)

C:\Users\skkuj\Anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: overflow encountered in exp


0.8245614035087719